In [1]:
import hashlib
import random

In [2]:
global max_bit 
max_bit = 3
max_node = 2**max_bit

In [3]:
def gethash(key):
    key = key.encode()
    h = hashlib.sha1(key)
    return int(h.hexdigest(),16)%max_node

In [4]:
key_list=[]
for i in range(max_node):
    h = gethash("%s"%i)
    key_list.append(h)

In [5]:
class ChordNode:
    def __init__(self,id):
        self.id = gethash(id)
        self.successor = None
        self.predecessor = None
        self.finger_table = {}
        self.key = []
        
    def create(self):
        self.predecessor = None
        self.successor = self
        for i in range(0,max_bit):
            self.finger_table[i] = self.successor
    
    def find_successor(self,keyid):#find keyid successor
        if keyid == self.id: 
            #case 1:key_id = node_id
            return self
        if self == self.successor: 
            #case 2:only one node
            return self
        if self.successor.id >= keyid and self.id < keyid:
            #case 3:node_id<key_id<successor_id
            return self.successor
        if self.id > self.successor.id and keyid > self.id:
            #case 4:successor_id<node_id<key_id
            return self.successor
        if self.id > self.successor.id and keyid <= self.successor.id:
            #case 5:key_id<successor_id<self.id
            return self.successor
        else:
            newNode = self.find_predecessor(keyid)
            if newNode == self:
                return newNode.successor.find_successor(keyid)
            else:
                return newNode.find_successor(keyid)
    
    def find_predecessor(self,keyid):
        for i in range(max_bit-1,-1,-1):
            if self.finger_table[i].id >= self.id and self.finger_table[i].id < keyid:
                return self.finger_table[i]      
        return self

    def join(self,randNode):
        #randNode is already in ChordRing, add a new node with the help of randNode
        self.predecessor = None
        self.successor = randNode.find_successor(self.id)
        for i in range(0,max_bit):
            self.finger_table[i] = self.successor

    def stabilize(self):
        x = self.successor.predecessor
        if x != None:
            if self == self.successor or (x.id > self.id and x.id < self.successor.id):
                self.successor = x
        self.successor.notify(self)
        
    def notify(self,node):
        if (self.predecessor == None) or (node.id > self.predecessor.id and node.id < self.id):
            self.predecessor = node
    
    def fix_fingers(self):
        for i in range(0,max_bit):
            successor = (self.id + 2**i)%2**i
            self.finger_table[i] = self.find_successor(successor)     

In [6]:
node3 = ChordNode('19')
node2 = ChordNode('889u90')
node1 = ChordNode('25099999')

In [7]:
print(node1.id,node2.id,node3.id)

1 4 7


In [8]:
node1.create()
node1.finger_table

{0: <__main__.ChordNode at 0x7f88f886e5f8>,
 1: <__main__.ChordNode at 0x7f88f886e5f8>,
 2: <__main__.ChordNode at 0x7f88f886e5f8>}

In [11]:
node2.join(node1)
node2.stabilize()
node2.fix_fingers()

In [12]:
node2

In [13]:
node2.predecessor

In [14]:
node2.successor

In [15]:
node1.predecessor